In [24]:
import os
import sys

src_path = r"D:\SEM 4\CS516\516 Project\src"
if src_path not in sys.path:
    sys.path.append(src_path)

In [25]:
import pandas as pd
import numpy as np
import shap

from sklearn.metrics import mean_absolute_error, accuracy_score
from scipy.stats import pearsonr

from data_loader import load_dataset
from modeling import train_smote_forest, evaluate_model, train_random_forest
from fairness_metrics import print_group_rates, disparate_impact, equal_opportunity
from preprocess import apply_reweighing

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import MetricFrame, selection_rate
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds

from lime.lime_tabular import LimeTabularExplainer

In [26]:
# Load data
df = load_dataset("../data/cleaned_dataset.csv")

# Pre-Processing

Using a preprocessing mitigation strategy - AIF360’s Reweighing

In [28]:
# Step 1: Apply AIF360 Reweighing (only used for fairness metrics, not for training if SMOTE is applied)
df = apply_reweighing(df, protected_attr='Gender', label_col='Employment')

gender_map = {0: 'Man', 1: 'NonBinary', 2: 'Woman'}
#age_map = {0: '<35', 1: '>35'}

df['Gender_group'] = df['Gender'].map(gender_map)
df['Age_group'] = df['Age']  


print(df['Gender_group'].unique())
print(df['Gender_group'].dtype)

print(df['Age_group'].unique())
print(df['Age_group'].dtype)


# Step 2: Feature engineering for modeling
features = [
    'Gender_group', 'Age_group', 'EdLevel', 'Gender_encoded', 'Age_encoded',
    'YearsCode', 'YearsCodePro', 'ComputerSkills', 'PreviousSalary'
]
X = pd.get_dummies(df[features], drop_first=True)
y = df['Employment'].astype(int)

print(X.columns)

['Man' 'Woman' 'NonBinary']
object
['<35' '>35']
object
Index(['Gender_encoded', 'Age_encoded', 'YearsCode', 'YearsCodePro',
       'ComputerSkills', 'PreviousSalary', 'Gender_group_NonBinary',
       'Gender_group_Woman', 'Age_group_>35', 'EdLevel_NoHigherEd',
       'EdLevel_Other', 'EdLevel_PhD', 'EdLevel_Undergraduate'],
      dtype='object')


In [29]:
# Step 3: Choose model strategy
use_smote = True  # Toggle this to False to use reweighing instead of SMOTE

if use_smote:
    model, X_test, y_test, y_pred = train_smote_forest(X, y)
else:
    sample_weights = df['instance_weight']
    model, X_test, y_test, y_pred = train_random_forest(X, y, sample_weights)

# Step 4: Evaluate base model performance (before fairness post-processing)
evaluate_model(y_test, y_pred)

# --- 3. Equal Accuracy by Gender Group ---
y_test_aligned = y_test.reset_index(drop=True)
y_pred_aligned = pd.Series(y_pred).reset_index(drop=True)
gender_column = df.loc[y_test.index, 'Gender_group'].reset_index(drop=True)

# --- 1. Pearson's Correlation ---
pearson_corr, _ = pearsonr(y_test, y_pred)
print(f"Pearson Correlation: {round(pearson_corr, 4)}")

# --- 2. Mean Absolute Error ---
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {round(mae, 4)}")

equal_accuracy = {}
for group in sorted(gender_column.unique()):
    group_mask = gender_column == group
    acc = accuracy_score(y_test_aligned[group_mask], y_pred_aligned[group_mask])
    equal_accuracy[group] = acc
    print(f"Accuracy for Gender {group}: {round(acc, 4)}")

equal_accuracy_gap = max(equal_accuracy.values()) - min(equal_accuracy.values())
print(f"Equal Accuracy Gap: {round(equal_accuracy_gap, 4)}")

Accuracy: 0.8051181995553337
Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.19      0.18      2571
           1       0.89      0.89      0.89     19468

    accuracy                           0.81     22039
   macro avg       0.54      0.54      0.54     22039
weighted avg       0.81      0.81      0.81     22039

Pearson Correlation: 0.0743
Mean Absolute Error: 0.1949
Accuracy for Gender Man: 0.8001
Accuracy for Gender NonBinary: 0.8318
Accuracy for Gender Woman: 0.8947
Equal Accuracy Gap: 0.0946


In [30]:
# Step 5: Custom fairness metrics based on fairness-aware predictions
print_group_rates(df, 'Gender')
print_group_rates(df, 'Age')
print_group_rates(df, 'EdLevel')

# Disparate impact comparisons
disparate_impact(df, 2, 0, 'Gender')  # Woman vs Man
disparate_impact(df, 1, 0, 'Gender')  # NonBinary vs Man
disparate_impact(df, '>35', '<35', 'Age')

# Equal opportunity using fairness-aware predictions
equal_opportunity(
    y_test.reset_index(drop=True),
    pd.Series(y_pred), 
    [0, 1, 2],  # Man, NonBinary, Woman
    'Gender',
    df.reset_index(drop=True)
)



Selection Rates by Gender:
0: 0.88
2: 0.91
1: 0.87

Selection Rates by Age:
<35: 0.90
>35: 0.85

Selection Rates by EdLevel:
Master: 0.88
Undergraduate: 0.90
PhD: 0.90
Other: 0.84
NoHigherEd: 0.80

Disparate Impact (2/0): 1.03

Disparate Impact (1/0): 0.98

Disparate Impact (>35/<35): 0.95

Equal Opportunity by group:
0: TPR = 0.89
1: TPR = 0.89
2: TPR = 0.87


# In-processing

In [31]:
def run_consistent_metrics(y_test, y_pred, sensitive_col='Gender_encoded', label_name='Gender'):
    """
    Evaluate predictions using MAE, Pearson correlation, and group-wise accuracy.
    Assumes access to global df and X_test_pmute for index alignment.
    """
    # Align predictions and labels
    y_test = y_test.reset_index(drop=True)
    y_pred = pd.Series(y_pred).reset_index(drop=True)
    
    # Align sensitive attribute values from df using test indices
    sensitive_series = df.loc[y_test.index, sensitive_col].reset_index(drop=True)

    # --- 1. Pearson Correlation ---
    pearson_corr, _ = pearsonr(y_test, y_pred)
    print(f"\nPearson Correlation: {round(pearson_corr, 4)}")

    # --- 2. Mean Absolute Error ---
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error: {round(mae, 4)}")

    # --- 3. Group-wise Accuracy ---
    print(f"\nAccuracy by {label_name} group:")
    for group in sorted(sensitive_series.unique()):
        group_mask = sensitive_series == group
        acc = accuracy_score(y_test[group_mask], y_pred[group_mask])
        print(f"{label_name} {group}: {round(acc, 4)}")


In [32]:
# --- 1. Prepare data: remove sensitive feature from X, pass separately ---
sensitive_feature_test = df.loc[X_test.index, "Gender_encoded"]
sensitive_feature_train = df.loc[X.index.difference(X_test.index), "Gender_encoded"]

X_train_eg = X.loc[X.index.difference(X_test.index)].drop(columns=["Gender_encoded"])
y_train_eg = y.loc[X.index.difference(X_test.index)]

X_test_eg = X_test.drop(columns=["Gender_encoded"])

# --- 2. Train fairness-aware model ---
eg_model = ExponentiatedGradient(
    estimator=LogisticRegression(solver="liblinear", class_weight='balanced'),
    constraints=EqualizedOdds(),
    eps=0.01
)
eg_model.fit(X_train_eg, y_train_eg, sensitive_features=sensitive_feature_train)

# --- 3. Predict ---
y_pred_eg = eg_model.predict(X_test_eg)

# --- 4. Evaluate performance ---
print("Exponentiated Gradient on SMOTE-balanced data:")
print("Accuracy:", accuracy_score(y_test, y_pred_eg))
print("Classification Report:")
print(classification_report(y_test, y_pred_eg))


Exponentiated Gradient on SMOTE-balanced data:
Accuracy: 0.516947229910613
Classification Report:
              precision    recall  f1-score   support

           0       0.13      0.54      0.21      2571
           1       0.89      0.51      0.65     19468

    accuracy                           0.52     22039
   macro avg       0.51      0.53      0.43     22039
weighted avg       0.80      0.52      0.60     22039



In [33]:
# --- 5. Custom evaluation function (consistent with pre-processing) ---
evaluate_model(y_test, y_pred_eg)

# --- 6. Reusable consistency metrics: Pearson, MAE, group-wise accuracy ---
run_consistent_metrics(y_test, y_pred_eg, sensitive_col='Gender_encoded', label_name='Gender')
run_consistent_metrics(y_test, y_pred_eg, sensitive_col='Age_encoded', label_name='Age')
run_consistent_metrics(y_test, y_pred_eg, sensitive_col='EdLevel', label_name='EdLevel')



Accuracy: 0.516947229910613
Classification Report:
               precision    recall  f1-score   support

           0       0.13      0.54      0.21      2571
           1       0.89      0.51      0.65     19468

    accuracy                           0.52     22039
   macro avg       0.51      0.53      0.43     22039
weighted avg       0.80      0.52      0.60     22039


Pearson Correlation: 0.0326
Mean Absolute Error: 0.4831

Accuracy by Gender group:
Gender 0: 0.5161
Gender 1: 0.5665
Gender 2: 0.5148

Pearson Correlation: 0.0326
Mean Absolute Error: 0.4831

Accuracy by Age group:
Age 0: 0.5145
Age 1: 0.5221

Pearson Correlation: 0.0326
Mean Absolute Error: 0.4831

Accuracy by EdLevel group:
EdLevel Master: 0.5159
EdLevel NoHigherEd: 0.5274
EdLevel Other: 0.5097
EdLevel PhD: 0.5078
EdLevel Undergraduate: 0.5192


In [34]:
# --- 7. Group fairness metrics ---
# Gender
mf_eg_gender = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test,
    y_pred=y_pred_eg,
    sensitive_features=sensitive_feature_test
)
print("\nFairness metrics by Gender:")
print(mf_eg_gender.by_group)

# Age
mf_eg_age = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test,
    y_pred=y_pred_eg,
    sensitive_features=df.loc[X_test.index, 'Age_encoded']
)
print("\nFairness metrics by Age:")
print(mf_eg_age.by_group)

# EdLevel
mf_eg_edlevel = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test,
    y_pred=y_pred_eg,
    sensitive_features=df.loc[X_test.index, 'EdLevel']
)
print("\nFairness metrics by EdLevel:")
print(mf_eg_edlevel.by_group)


Fairness metrics by Gender:
                accuracy  selection_rate
Gender_encoded                          
0               0.516294        0.507212
1               0.523697        0.552133
2               0.527290        0.515595

Fairness metrics by Age:
             accuracy  selection_rate
Age_encoded                          
0            0.538333        0.541675
1            0.476947        0.446340

Fairness metrics by EdLevel:
               accuracy  selection_rate
EdLevel                                
Master         0.516273        0.513770
NoHigherEd     0.508676        0.516895
Other          0.497835        0.470934
PhD            0.508951        0.478261
Undergraduate  0.524083        0.517819


In [35]:
# --- 8. Custom fairness functions ---
print_group_rates(df, 'Gender')
print_group_rates(df, 'Age')
print_group_rates(df, 'EdLevel')

disparate_impact(df, 2, 0, 'Gender')  # Woman vs Man
disparate_impact(df, 1, 0, 'Gender')  # NonBinary vs Man
disparate_impact(df, '>35', '<35', 'Age')

equal_opportunity(
    y_test.reset_index(drop=True),
    pd.Series(y_pred_eg),
    [0, 1, 2],  # Gender groups
    'Gender',
    df.reset_index(drop=True)
)



Selection Rates by Gender:
0: 0.88
2: 0.91
1: 0.87

Selection Rates by Age:
<35: 0.90
>35: 0.85

Selection Rates by EdLevel:
Master: 0.88
Undergraduate: 0.90
PhD: 0.90
Other: 0.84
NoHigherEd: 0.80

Disparate Impact (2/0): 1.03

Disparate Impact (1/0): 0.98

Disparate Impact (>35/<35): 0.95

Equal Opportunity by group:
0: TPR = 0.51
1: TPR = 0.57
2: TPR = 0.51


# Post Processing

In [36]:
# --- 1. 3-Way Split: Train (60%), Validation (10%), Test (30%) ---
X_trainval, X_test_post, y_trainval, y_test_post = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

X_train_post, X_val_post, y_train_post, y_val_post = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=42
)

# --- 2. Apply SMOTE to training data ---
smote = SMOTE(random_state=42)
X_train_post_smote, y_train_post_smote = smote.fit_resample(X_train_post, y_train_post)

# --- 3. Train classifier on SMOTE-balanced data ---
model_post = RandomForestClassifier(n_estimators=100, random_state=42)
model_post.fit(X_train_post_smote, y_train_post_smote)

# --- 4. Apply ThresholdOptimizer using validation set ---
sensitive_gender_val = df.loc[X_val_post.index, "Gender_encoded"]

postprocessor = ThresholdOptimizer(
    estimator=model_post,
    constraints="equalized_odds",
    predict_method="predict_proba"
)

postprocessor.fit(
    X_val_post,
    y_val_post,
    sensitive_features=sensitive_gender_val
)

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=RandomForestClassifier(random_state=42),
                   predict_method='predict_proba')

In [37]:
# --- 5. Predict on test set ---
sensitive_gender_test = df.loc[X_test_post.index, "Gender_encoded"]
y_pred_fair = postprocessor.predict(
    X_test_post,
    sensitive_features=sensitive_gender_test
)

# --- 6. Evaluate overall performance ---
print("\nFairness-aware Evaluation (SMOTE + Equalized Odds):")
print("Overall Accuracy:", accuracy_score(y_test_post, y_pred_fair))
print("Classification Report:")
print(classification_report(y_test_post, y_pred_fair))

# --- 7. Custom Evaluation ---
evaluate_model(y_test_post, y_pred_fair)

run_consistent_metrics(y_test_post, y_pred_fair, sensitive_col='Gender_encoded', label_name='Gender')
run_consistent_metrics(y_test_post, y_pred_fair, sensitive_col='Age_encoded', label_name='Age')
run_consistent_metrics(y_test_post, y_pred_fair, sensitive_col='EdLevel', label_name='EdLevel')


Fairness-aware Evaluation (SMOTE + Equalized Odds):
Overall Accuracy: 0.8660556286582876
Classification Report:
              precision    recall  f1-score   support

           0       0.23      0.06      0.10      2576
           1       0.89      0.97      0.93     19463

    accuracy                           0.87     22039
   macro avg       0.56      0.52      0.51     22039
weighted avg       0.81      0.87      0.83     22039

Accuracy: 0.8660556286582876
Classification Report:
               precision    recall  f1-score   support

           0       0.23      0.06      0.10      2576
           1       0.89      0.97      0.93     19463

    accuracy                           0.87     22039
   macro avg       0.56      0.52      0.51     22039
weighted avg       0.81      0.87      0.83     22039


Pearson Correlation: 0.0635
Mean Absolute Error: 0.1339

Accuracy by Gender group:
Gender 0: 0.865
Gender 1: 0.8484
Gender 2: 0.8996

Pearson Correlation: 0.0635
Mean Absolute Err

In [38]:
# --- 8. MetricFrame Fairness Evaluation ---
mf_post_gender = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test_post,
    y_pred=y_pred_fair,
    sensitive_features=sensitive_gender_test
)
print("\nFairness metrics by Gender:")
print(mf_post_gender.by_group)

mf_post_age = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test_post,
    y_pred=y_pred_fair,
    sensitive_features=df.loc[X_test_post.index, 'Age_encoded']
)
print("\nFairness metrics by Age:")
print(mf_post_age.by_group)

mf_post_edlevel = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test_post,
    y_pred=y_pred_fair,
    sensitive_features=df.loc[X_test_post.index, 'EdLevel']
)
print("\nFairness metrics by EdLevel:")
print(mf_post_edlevel.by_group)


Fairness metrics by Gender:
                accuracy  selection_rate
Gender_encoded                          
0               0.864945        0.968736
1               0.861575        0.966587
2               0.890304        0.962782

Fairness metrics by Age:
             accuracy  selection_rate
Age_encoded                          
0            0.890317        0.981111
1            0.821749        0.945242

Fairness metrics by EdLevel:
               accuracy  selection_rate
EdLevel                                
Master         0.873960        0.975217
NoHigherEd     0.784710        0.936731
Other          0.798713        0.927696
PhD            0.879423        0.982962
Undergraduate  0.888998        0.979065


In [39]:
# --- 9. Custom Fairness Metrics ---
print_group_rates(df, 'Gender')
print_group_rates(df, 'Age')
print_group_rates(df, 'EdLevel')

disparate_impact(df, 2, 0, 'Gender')  # Woman vs Man
disparate_impact(df, 1, 0, 'Gender')  # NonBinary vs Man
disparate_impact(df, '>35', '<35', 'Age')

equal_opportunity(
    y_test_post.reset_index(drop=True),
    pd.Series(y_pred_fair),
    [0, 1, 2],  # Gender groups
    'Gender',
    df.reset_index(drop=True)
)



Selection Rates by Gender:
0: 0.88
2: 0.91
1: 0.87

Selection Rates by Age:
<35: 0.90
>35: 0.85

Selection Rates by EdLevel:
Master: 0.88
Undergraduate: 0.90
PhD: 0.90
Other: 0.84
NoHigherEd: 0.80

Disparate Impact (2/0): 1.03

Disparate Impact (1/0): 0.98

Disparate Impact (>35/<35): 0.95

Equal Opportunity by group:
0: TPR = 0.97
1: TPR = 0.97
2: TPR = 0.98


# Explainability

## a) Proxy Mute

In [40]:
# --- 1. Train-Test Split ---
X_train_pmute, X_test_pmute, y_train_pmute, y_test_pmute = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# --- 2. Train Base Model (Logistic Regression with Class Weights) ---
lr_model_base = LogisticRegression(solver="liblinear", class_weight='balanced')
lr_model_base.fit(X_train_pmute, y_train_pmute)

# --- 3. SHAP Explainability (KernelExplainer for probability output) ---
explainer = shap.KernelExplainer(
    lr_model_base.predict_proba,
    X_train_pmute.sample(100, random_state=42)
)
shap_values = explainer.shap_values(X_test_pmute[:100])

# --- 4. Mean Absolute SHAP Importance ---
mean_abs_shap = np.abs(shap_values[1]).mean(axis=0)
shap_summary = pd.DataFrame({
    "feature": X_train_pmute.columns,
    "mean_abs_shap": mean_abs_shap
}).sort_values(by="mean_abs_shap", ascending=False)

  0%|          | 0/100 [00:00<?, ?it/s]

In [41]:
# --- 5. Define Proxy Features to Mute (based on SHAP summary) ---
proxy_features = ['PreviousSalary', 'EdLevel_Undergraduate', 'ComputerSkills']

# --- 6. Muting Proxy Features in Test Set ---
X_test_muted = X_test_pmute.copy()
for col in proxy_features:
    if col in X_test_muted.columns:
        X_test_muted[col] = X_test_muted[col].mean()

# --- 7. Predict on Muted Test Set ---
y_pred_muted = lr_model_base.predict(X_test_muted)

# --- 8. Performance Metrics ---
print("ProxyMute (Revised Proxy List):")
print("Accuracy:", accuracy_score(y_test_pmute, y_pred_muted))
print("Classification Report:")
print(classification_report(y_test_pmute, y_pred_muted))


ProxyMute (Revised Proxy List):
Accuracy: 0.6645038341122556
Classification Report:
              precision    recall  f1-score   support

           0       0.16      0.44      0.23      2576
           1       0.90      0.69      0.79     19463

    accuracy                           0.66     22039
   macro avg       0.53      0.57      0.51     22039
weighted avg       0.82      0.66      0.72     22039



In [42]:
# --- 9. Fairness Evaluation Using MetricFrame ---
# Gender
mf_shap_gender = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test_pmute,
    y_pred=y_pred_muted,
    sensitive_features=df.loc[X_test_pmute.index, "Gender_encoded"]
)
print("\nFairness metrics by Gender (ProxyMute SHAP):")
print(mf_shap_gender.by_group)

# Age
mf_shap_age = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test_pmute,
    y_pred=y_pred_muted,
    sensitive_features=df.loc[X_test_pmute.index, "Age_encoded"]
)
print("\nFairness metrics by Age (ProxyMute SHAP):")
print(mf_shap_age.by_group)

# EdLevel
mf_shap_edlevel = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
    y_true=y_test_pmute,
    y_pred=y_pred_muted,
    sensitive_features=df.loc[X_test_pmute.index, "EdLevel"]
)
print("\nFairness metrics by EdLevel (ProxyMute SHAP):")
print(mf_shap_edlevel.by_group)



Fairness metrics by Gender (ProxyMute SHAP):
                accuracy  selection_rate
Gender_encoded                          
0               0.656197        0.669013
1               0.739857        0.749403
2               0.801175        0.857982

Fairness metrics by Age (ProxyMute SHAP):
             accuracy  selection_rate
Age_encoded                          
0            0.873675        0.959553
1            0.282508        0.167479

Fairness metrics by EdLevel (ProxyMute SHAP):
               accuracy  selection_rate
EdLevel                                
Master         0.663126        0.666313
NoHigherEd     0.596661        0.630931
Other          0.543199        0.537377
PhD            0.595020        0.570118
Undergraduate  0.712071        0.739421


In [43]:
# --- 10. Consistent Utility Metrics ---
evaluate_model(y_test_pmute, y_pred_muted)

run_consistent_metrics(y_test_pmute, y_pred_muted, sensitive_col='Gender_encoded', label_name='Gender')
run_consistent_metrics(y_test_pmute, y_pred_muted, sensitive_col='Age_encoded', label_name='Age')
run_consistent_metrics(y_test_pmute, y_pred_muted, sensitive_col='EdLevel', label_name='EdLevel')



Accuracy: 0.6645038341122556
Classification Report:
               precision    recall  f1-score   support

           0       0.16      0.44      0.23      2576
           1       0.90      0.69      0.79     19463

    accuracy                           0.66     22039
   macro avg       0.53      0.57      0.51     22039
weighted avg       0.82      0.66      0.72     22039


Pearson Correlation: 0.0904
Mean Absolute Error: 0.3355

Accuracy by Gender group:
Gender 0: 0.6652
Gender 1: 0.6064
Gender 2: 0.673

Pearson Correlation: 0.0904
Mean Absolute Error: 0.3355

Accuracy by Age group:
Age 0: 0.6637
Age 1: 0.6662

Pearson Correlation: 0.0904
Mean Absolute Error: 0.3355

Accuracy by EdLevel group:
EdLevel Master: 0.6585
EdLevel NoHigherEd: 0.6559
EdLevel Other: 0.6643
EdLevel PhD: 0.633
EdLevel Undergraduate: 0.6704


In [44]:
# --- 11. Custom Fairness Metrics ---
print_group_rates(df, 'Gender')
print_group_rates(df, 'Age')
print_group_rates(df, 'EdLevel')

disparate_impact(df, 2, 0, 'Gender')  # Woman vs Man
disparate_impact(df, 1, 0, 'Gender')  # NonBinary vs Man
disparate_impact(df, '>35', '<35', 'Age')

equal_opportunity(
    y_test_pmute.reset_index(drop=True),
    pd.Series(y_pred_muted),
    [0, 1, 2],  # Gender groups
    'Gender',
    df.reset_index(drop=True)
)


Selection Rates by Gender:
0: 0.88
2: 0.91
1: 0.87

Selection Rates by Age:
<35: 0.90
>35: 0.85

Selection Rates by EdLevel:
Master: 0.88
Undergraduate: 0.90
PhD: 0.90
Other: 0.84
NoHigherEd: 0.80

Disparate Impact (2/0): 1.03

Disparate Impact (1/0): 0.98

Disparate Impact (>35/<35): 0.95

Equal Opportunity by group:
0: TPR = 0.70
1: TPR = 0.64
2: TPR = 0.69


## b) Refined Proxy Mute

In [45]:
# --- 1. Initialize LIME Explainer ---
explainer = LimeTabularExplainer(
    training_data=X_train_pmute.values,
    feature_names=X_train_pmute.columns.tolist(),
    class_names=["Not Employed", "Employed"],
    mode="classification",
    discretize_continuous=False
)

# --- 2. Local Muting: Top 2 Features per Instance ---
X_test_localmute_muted = X_test_pmute.copy()

for i in range(500):  # Apply LIME only to top 500 for speed
    exp = explainer.explain_instance(
        X_test_pmute.iloc[i].values,
        lambda x: lr_model_base.predict_proba(pd.DataFrame(x, columns=X_train_pmute.columns)),
        num_features=2
    )
    top_features = [f[0] for f in exp.as_list()]
    
    for f in top_features:
        f_name = f.split('<')[0].split('>')[0].split('=')[0].strip()
        if f_name in X_test_localmute_muted.columns:
            col_idx = X_test_localmute_muted.columns.get_loc(f_name)
            mean_val = X_train_pmute[f_name].mean()
            col_dtype = X_test_localmute_muted.dtypes[f_name]
            X_test_localmute_muted.iat[i, col_idx] = col_dtype.type(mean_val)

# --- 3. Predict ---
y_pred_lime_localmute = lr_model_base.predict(X_test_localmute_muted)

In [46]:
# --- 4. Performance Metrics ---
print("Refined ProxyMute (LIME, Top 2 Features):")
print("Accuracy:", accuracy_score(y_test_pmute, y_pred_lime_localmute))
print("Classification Report:")
print(classification_report(y_test_pmute, y_pred_lime_localmute))

# --- 5. Fairness: MetricFrame by Group ---
for attr, label in [("Gender_encoded", "Gender"), ("Age_encoded", "Age"), ("EdLevel", "EdLevel")]:
    mf = MetricFrame(
        metrics={"accuracy": accuracy_score, "selection_rate": selection_rate},
        y_true=y_test_pmute,
        y_pred=y_pred_lime_localmute,
        sensitive_features=df.loc[X_test_pmute.index, attr]
    )
    print(f"\nFairness metrics by {label}:")
    print(mf.by_group)

Refined ProxyMute (LIME, Top 2 Features):
Accuracy: 0.6051544988429602
Classification Report:
              precision    recall  f1-score   support

           0       0.16      0.58      0.25      2576
           1       0.92      0.61      0.73     19463

    accuracy                           0.61     22039
   macro avg       0.54      0.59      0.49     22039
weighted avg       0.83      0.61      0.68     22039


Fairness metrics by Gender:
                accuracy  selection_rate
Gender_encoded                          
0               0.595466        0.574882
1               0.625298        0.634845
2               0.792360        0.811949

Fairness metrics by Age:
             accuracy  selection_rate
Age_encoded                          
0            0.710343        0.725581
1            0.413055        0.333932

Fairness metrics by EdLevel:
               accuracy  selection_rate
EdLevel                                
Master         0.485219        0.439193
NoHigherEd     0.

In [47]:
# --- 6. Custom Evaluation ---
evaluate_model(y_test_pmute, y_pred_lime_localmute)

run_consistent_metrics(y_test_pmute, y_pred_lime_localmute, sensitive_col='Gender_encoded', label_name='Gender')
run_consistent_metrics(y_test_pmute, y_pred_lime_localmute, sensitive_col='Age_encoded', label_name='Age')
run_consistent_metrics(y_test_pmute, y_pred_lime_localmute, sensitive_col='EdLevel', label_name='EdLevel')

Accuracy: 0.6051544988429602
Classification Report:
               precision    recall  f1-score   support

           0       0.16      0.58      0.25      2576
           1       0.92      0.61      0.73     19463

    accuracy                           0.61     22039
   macro avg       0.54      0.59      0.49     22039
weighted avg       0.83      0.61      0.68     22039


Pearson Correlation: 0.1217
Mean Absolute Error: 0.3948

Accuracy by Gender group:
Gender 0: 0.6055
Gender 1: 0.5851
Gender 2: 0.6057

Pearson Correlation: 0.1217
Mean Absolute Error: 0.3948

Accuracy by Age group:
Age 0: 0.6025
Age 1: 0.6107

Pearson Correlation: 0.1217
Mean Absolute Error: 0.3948

Accuracy by EdLevel group:
EdLevel Master: 0.6052
EdLevel NoHigherEd: 0.6119
EdLevel Other: 0.5968
EdLevel PhD: 0.5889
EdLevel Undergraduate: 0.608


In [48]:
# --- 7. Custom Fairness Functions ---
print_group_rates(df, 'Gender')
print_group_rates(df, 'Age')
print_group_rates(df, 'EdLevel')

disparate_impact(df, 2, 0, 'Gender')
disparate_impact(df, 1, 0, 'Gender')
disparate_impact(df, '>35', '<35', 'Age')

equal_opportunity(
    y_test_pmute.reset_index(drop=True),
    pd.Series(y_pred_lime_localmute),
    [0, 1, 2],
    'Gender',
    df.reset_index(drop=True)
)


Selection Rates by Gender:
0: 0.88
2: 0.91
1: 0.87

Selection Rates by Age:
<35: 0.90
>35: 0.85

Selection Rates by EdLevel:
Master: 0.88
Undergraduate: 0.90
PhD: 0.90
Other: 0.84
NoHigherEd: 0.80

Disparate Impact (2/0): 1.03

Disparate Impact (1/0): 0.98

Disparate Impact (>35/<35): 0.95

Equal Opportunity by group:
0: TPR = 0.61
1: TPR = 0.59
2: TPR = 0.61
